## Transient Object Removal from LiDAR Point Clouds Using Octree-Based KNN Filtering
*Created by: Divya Natekar (dyn2009), NYU CUSP: Summer Guided Research*
#### Program Logic:
This program implements a streamlined point-cloud filtering workflow by generating a reduced training set from the original jay-willoughby.ply file through octree downsampling. Each occupied octree cell is represented by its centroid, producing a smaller yet structurally representative dataset. A KD-Tree is then constructed on this reduced set to make KNN searches faster and less memory-demanding.

For the filtering step, the algorithm mimics a scanner positioned at each of the four corners. For every point in the cloud, it defines a search radius that increases with distance from the scanner and checks how many downsampled training points lie within that radius. Points with counts below a set threshold are classified as noise and removed, while the remaining points are passed on to the next corner for further refinement.

In [1]:
import open3d as o3d, numpy as np, os, math, random, pathlib

inp = "jay-willoughby.ply"
noise_ref = "jw-noise-points.ply"
ideal_ref = "manual-filtering.ply"
outdir = "out_octree_knn_progressive_calibrated"
os.makedirs(outdir, exist_ok=True)

pcd = o3d.io.read_point_cloud(inp)
pts = np.asarray(pcd.points).astype(np.float64)
cols = (np.asarray(pcd.colors).astype(np.float64) if pcd.has_colors() else np.zeros((len(pts),3)))
if cols.size>0 and cols.max()>1.0: cols = cols/255.0

bbox = pcd.get_axis_aligned_bounding_box()
mn, mx = bbox.get_min_bound(), bbox.get_max_bound()
z0 = float(mn[2])
corners = np.array([[mn[0],mn[1],z0],[mx[0],mn[1],z0],[mx[0],mx[1],z0],[mn[0],mx[1],z0]],dtype=np.float64)
span = (mx - mn) + 1e-9
scene_diag = float(np.linalg.norm(mx - mn))

def write_empty_ply(path):
    with open(path, "w") as f:
        f.write("ply\nformat ascii 1.0\n")
        f.write("element vertex 0\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        f.write("end_header\n")

def write_ply(path, p, c):
    if p.size==0:
        write_empty_ply(path); return
    q = o3d.geometry.PointCloud()
    q.points = o3d.utility.Vector3dVector(p.astype(np.float64))
    if c is not None and c.size==p.shape[0]*3:
        r = c
        if r.size>0 and r.max()<=1.0: r = np.clip(r,0,1)
        if r.size>0: q.colors = o3d.utility.Vector3dVector(r.astype(np.float64))
    o3d.io.write_point_cloud(path, q, write_ascii=True)

def octree_downsample(points, colors, depth):
    res = 1<<depth
    q = np.floor((points - mn)/span*res).astype(np.int64)
    q = np.clip(q,0,res-1)
    keys = q[:,0] + res*q[:,1] + (res**2)*q[:,2]
    order = np.argsort(keys)
    k_sorted = keys[order]
    p_sorted = points[order]
    c_sorted = colors[order] if colors is not None else None
    boundaries = np.flatnonzero(np.concatenate(([True], k_sorted[1:]!=k_sorted[:-1], [True])))
    counts = np.diff(boundaries); starts = boundaries[:-1]
    centroids = np.add.reduceat(p_sorted, starts, axis=0) / counts[:,None]
    if c_sorted is not None:
        c_means = np.add.reduceat(c_sorted, starts, axis=0) / counts[:,None]
    else:
        c_means = np.zeros((len(centroids),3))
    voxel_len = span/float(res)
    vox_mean = float(np.mean(voxel_len))
    return centroids, c_means, vox_mean

target_train_min = max(60000, len(pts)//35)
target_train_max = max(160000, len(pts)//7)
best = None
for d in range(6,11):
    tr_pts, tr_cols, vox_len = octree_downsample(pts, cols, d)
    if best is None: best = (d, tr_pts, tr_cols, vox_len)
    if target_train_min <= len(tr_pts) <= target_train_max:
        best = (d, tr_pts, tr_cols, vox_len)
        break
depth, train_pts, train_cols, vox_len = best

train_pcd = o3d.geometry.PointCloud()
train_pcd.points = o3d.utility.Vector3dVector(train_pts)
kdt = o3d.geometry.KDTreeFlann(train_pcd)

base_r = max(vox_len*2.0, scene_diag/900.0)

def score_density(points, corner, r_scale, alpha, chunk=150000):
    v = points - corner
    d = np.linalg.norm(v, axis=1); d = np.maximum(d, 1e-6)
    r = base_r*r_scale + alpha*d
    dens = np.zeros(len(points), dtype=np.float64)
    for i in range(0,len(points),chunk):
        p = points[i:i+chunk]; rr = r[i:i+chunk]
        local = np.zeros(len(p), dtype=np.float64)
        for j,x in enumerate(p):
            _, idx, _ = kdt.search_radius_vector_3d(o3d.utility.Vector3dVector([x])[0], float(rr[j]))
            vol = (4.0/3.0)*math.pi*(rr[j]**3)
            local[j] = (len(idx)/vol) if vol>0 else 0.0
        dens[i:i+chunk] = local
    return dens

def progressive_filter(cur_pts, cur_cols, params_four, chunk=200000):
    rem_pts, rem_cols = cur_pts, cur_cols
    all_outputs = []
    for ci in range(4):
        r_scale, alpha, pctl = params_four[ci]
        dens = score_density(rem_pts, corners[ci], r_scale, alpha, chunk=max(50000,chunk//2))
        tau = np.percentile(dens, pctl)
        keep = dens>=tau
        kept_pts = rem_pts[keep]
        kept_cols = rem_cols[keep] if rem_cols is not None and rem_cols.size>0 else np.zeros((len(kept_pts),3))
        rmv_pts = rem_pts[~keep]
        rmv_cols = rem_cols[~keep] if rem_cols is not None and rem_cols.size>0 else np.zeros((len(rmv_pts),3))
        all_outputs.append((kept_pts, kept_cols, rmv_pts, rmv_cols))
        rem_pts, rem_cols = kept_pts, kept_cols
    return rem_pts, rem_cols, all_outputs

def calibrate_params():
    if not (os.path.exists(noise_ref) and os.path.exists(ideal_ref)): 
        return None
    p_noise = o3d.io.read_point_cloud(noise_ref)
    p_ideal = o3d.io.read_point_cloud(ideal_ref)
    n_pts = np.asarray(p_noise.points).astype(np.float64)
    i_pts = np.asarray(p_ideal.points).astype(np.float64)
    if len(n_pts)==0 or len(i_pts)==0:
        return None
    N = min(80000, len(pts))
    idx = np.random.choice(len(pts), size=N, replace=False)
    sample = pts[idx]
    n_kdt = o3d.geometry.KDTreeFlann(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n_pts)))
    i_kdt = o3d.geometry.KDTreeFlann(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(i_pts)))
    r_near = max(scene_diag/2000.0, base_r*0.75)
    y_noise = np.zeros(N, dtype=np.int8)
    y_ideal = np.zeros(N, dtype=np.int8)
    for j,x in enumerate(sample):
        _, n_idx, _ = n_kdt.search_radius_vector_3d(o3d.utility.Vector3dVector([x])[0], float(r_near))
        _, i_idx, _ = i_kdt.search_radius_vector_3d(o3d.utility.Vector3dVector([x])[0], float(r_near))
        y_noise[j] = 1 if len(n_idx)>0 else 0
        y_ideal[j] = 1 if len(i_idx)>0 else 0
    y_keep = ((y_ideal==1) & (y_noise==0)).astype(np.int8)

    r_scales = [1.05, 1.15, 1.25]
    alphas =  [0.0055, 0.0060, 0.0065]
    pcts   =  [30.0, 35.0, 40.0]
    best_params = None
    best_f1 = -1.0

    def f1_score(y_true, y_pred):
        tp = np.sum((y_true==1)&(y_pred==1))
        fp = np.sum((y_true==0)&(y_pred==1))
        fn = np.sum((y_true==1)&(y_pred==0))
        prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9)
        return 2*prec*rec/(prec+rec+1e-9)

    for r_scale in r_scales:
        for alpha in alphas:
            dens = score_density(sample, corners[0], r_scale, alpha, chunk=40000)
            for pctl in pcts:
                tau = np.percentile(dens, pctl)
                y_pred = (dens>=tau).astype(np.int8)
                f1 = f1_score(y_keep, y_pred)
                if f1>best_f1:
                    best_f1 = f1
                    base = (r_scale, alpha, pctl)
                    best_params = [base, base, base, base]

    return best_params

params_four = calibrate_params()
if params_four is None:
    params_four = [
        (1.20, 0.0060, 35.0),
        (1.20, 0.0060, 35.0),
        (1.25, 0.0060, 35.0),
        (1.30, 0.0060, 35.0),
    ]

cur_pts, cur_cols = pts, cols
for loop in range(1,5):
    rem_pts, rem_cols, outputs = progressive_filter(cur_pts, cur_cols, params_four)
    for cidx,(kept_pts, kept_cols, rmv_pts, rmv_cols) in enumerate(outputs, start=1):
        write_ply(os.path.join(outdir, f"L{loop}C{cidx}_removed.ply"), rmv_pts, rmv_cols)
        write_ply(os.path.join(outdir, f"L{loop}C{cidx}_remaining.ply"), kept_pts, kept_cols)
    cur_pts, cur_cols = rem_pts, rem_cols

print("done", outdir, "depth", depth, "train_size", len(train_pts), "final_remaining", len(cur_pts), "params", params_four)


done out_octree_knn_progressive_calibrated depth 8 train_size 182798 final_remaining 4523 params [(1.25, 0.0055, 30.0), (1.25, 0.0055, 30.0), (1.25, 0.0055, 30.0), (1.25, 0.0055, 30.0)]
